In [ ]:
import os
import json

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import confusion_matrix

In [ ]:
path = "/Users/tanny/Documents/Projects/CULI Scoring/saves/"
cefr_label_path = "/Users/tanny/Documents/Projects/CULI Scoring/main/assets/cefr_label.csv"

In [ ]:
metrics = []
predictions = []

for file in os.listdir(path):
    if file.endswith("metrics.json"):
        metrics.append(os.path.join(path, file))
    elif file.endswith("predictions.json"):
        predictions.append(os.path.join(path, file))

In [ ]:
def read_json(json_path) -> pd.DataFrame:
    with open(json_path, "r") as f:
        return pd.DataFrame(json.load(f))

def read_csv(csv_path) -> pd.DataFrame:
    return pd.read_csv(csv_path)

def visualize_metrics(metrics: str) -> None:
    plt.style.use('dark_background')
    data = read_json(metrics)
    # Plot Loss
    plt.figure(figsize=(10, 5))
    plt.plot(data["epoch"], data["train_loss"], label="Train Loss")
    plt.plot(data["epoch"], data["val_loss"], label="Val Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title(f"Loss Over Epochs: {os.path.basename(metrics)}")
    plt.legend()
    plt.show()
        # Plot Accuracy
    plt.figure(figsize=(10, 5))
    plt.plot(data["epoch"], data["train_acc"], label="Train Accuracy")
    plt.plot(data["epoch"], data["val_acc"], label="Val Accuracy")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.title(f"Accuracy Over Epochs: {os.path.basename(metrics)}")
    plt.legend()
    plt.show()

def get_max_score(metrics: str, column: str) -> float:
    max_score = 0.0
    data = read_json(metrics)
    if column in data.columns:
        max_score = max(max_score, data[column].max())
    return max_score

def visualize_confusion_matrix(predictions: str, true_col: str, pred_col: str, label_map: pd.DataFrame) -> None:
    data = read_json(predictions)

    key_to_label = dict(zip(label_map["key"], label_map["value"]))
    data[true_col] = data[true_col].map(key_to_label)
    data[pred_col] = data[pred_col].map(key_to_label)

    y_true = data[true_col]
    y_pred = data[pred_col]

    labels = list(label_map["value"])
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
    plt.title(f"Confusion Matrix for {true_col} vs {pred_col} in {os.path.basename(predictions)}")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.show()

In [ ]:
for i in metrics:
    visualize_metrics(i)
    max_score = get_max_score(i, "val_acc")
    print(f"Max validation accuracy for {os.path.basename(i)}: {max_score:.4f}")

In [ ]:
cefr_label_map = pd.DataFrame({
    "key": [0, 1, 2, 3],
    "value": ["A2_0", "B1_1", "B1_2", "B2_0"]
})

for i in predictions:
    visualize_confusion_matrix(i, "y_true", "y_pred", cefr_label_map)
    max_score = get_max_score(i, "y_pred")
    print(f"Max prediction score for {os.path.basename(i)}: {max_score:.4f}")